# 2. Tipos de dados, marcadores gráficos, e Canais de codificação visual

A visualização representa os dados utilizando uma coleção de "marcadores gráficos" (gráficos de barras, linhas, pontos, etc.). Os atributos do marcador &mdash; como posição, formato, tamanho ou cor &mdash; serve como "canais" por onde podemos codificar os dados subjacentes.

Utilizando um _framework_ básico de "tipos de dados", "marcadores", e "canais de codificação", é possível criar de maneira concisa uma grande variedade de visualizações. Nesse notebook, será explorado cada um desses elementos e será exibido como utilizar eles para criar um gráfico customizado oriundo de dados estatísticos.

Esse notebook é parte de: [data visualization curriculum](https://github.com/uwdata/visualization-curriculum).

In [1]:
import pandas as pd
import altair as alt

## 2.1. Dados de Desenvolvimento Global

Serão analisados alguns dados sobre a saúde global e sobre a população para um número de países, no período entre 1955 e 2005. Os dados foram coletados do site: [Gapminder Foundation](https://www.gapminder.org/) e publicado em [Hans Rosling's popular TED talk](https://www.youtube.com/watch?v=hVimVzgtD6w). Se você ainda não assistiu essa palestra, é recomendado que você assista primeiro.

Primeiramente serão carregados os dados de: [vega-datasets](https://github.com/vega/vega-datasets) da coleção do _dataframe_ pandas.

In [2]:
from vega_datasets import data as vega_data
data = vega_data.gapminder()

Qual o tamanho dos dados carregados?

In [3]:
data.shape

(693, 6)

693 linhas e 6 colunas. Será examinado o conteúdo dos dados.

In [4]:
data.head(5)

year      country  cluster       pop  life_expect  fertility
0  1955  Afghanistan        0   8891209       30.332        7.7
1  1960  Afghanistan        0   9829450       31.997        7.7
2  1965  Afghanistan        0  10997885       34.020        7.7
3  1970  Afghanistan        0  12430623       36.088        7.7
4  1975  Afghanistan        0  14132019       38.438        7.7

Para cada país em `country` e ano em `year` (no intervalo de 5 em 5 anos), foi medida a fertilidade em termos de numeros de filhos por mulher (`fertility`), expectativa de vida em anos (`life_expect`) e população total (`pop`).

Também será visto o campo `cluster` que é um código de um número inteiro. O que isso pode representar? Será resolvido esse "mistério" quando visualizarmos os dados.

É possível criar um _dataframe_ menor. Por exemplo é possível filtrar somente os valores dos anos 2000. Como pode ser visto abaixo:

In [5]:
data2000 = data.loc[data['year'] == 2000]

In [6]:
data2000.head(5)

year      country  cluster       pop  life_expect  fertility
9   2000  Afghanistan        0  23898198       42.129     7.4792
20  2000    Argentina        3  37497728       74.340     2.3500
31  2000        Aruba        3     69539       73.451     2.1240
42  2000    Australia        4  19164620       80.370     1.7560
53  2000      Austria        1   8113413       78.980     1.3820

## 2.2. Tipos de dados


O primeiro "ingrediente" na visualização efetiva são os dados de entrada. Os valores dos dados podem representar diferentes formas de medidas. O que será que as comparações dessas medidas sugerem? E quais codificações visuais apoiam essas comparações?

Serão analisados os tipos básicos de dados, que o Altair usa para informar suas escolhas de codificação visual. Esses tipos de dados determinam quais comparações são possíveis de fazer, e com isso guiam as decisões no _design_ da visualização.

### 2.2.1. Nominal (N)

Os dados *nominais* (também chamados de dados *categóricos*) consistem em nomes de categorias.

Com dados nominais, podemos comparar a igualdade de valores: *o valor A é igual ou diferente do valor B? (A = B)*, sustentando afirmações como “A é igual a B” ou “A não é igual a B”. No conjunto de dados acima, o campo **`country`** é nominal.

Ao visualizar dados nominais, devemos ser capazes de ver facilmente se os valores são iguais ou diferentes: posição, matiz de cor (azul, vermelho, verde, etc.) e forma podem ajudar. No entanto, usar um canal de tamanho para codificar dados nominais pode nos enganar, sugerindo diferenças de ordem ou magnitude entre valores que não existem!


### 2.2.2. Ordinal (O)

Dados *ordinais* consistem em valores que têm uma ordenação específica.

Com dados ordinais, podemos comparar a ordem de classificação dos valores: *o valor A vem antes ou depois do valor B? (A < B)*, sustentando afirmações como “A é menor que B” ou “A é maior que B”. No conjunto de dados acima, podemos tratar o campo **`year`** como ordinal.

Ao visualizar dados ordinais, devemos perceber um senso de ordem classificatória. Posição, tamanho ou valor de cor (brilho) podem ser apropriados, enquanto o matriz de cor (que não é perceptualmente ordenado) seria menos apropriado.


### 2.2.3. Quantitativo (Q)

Com dados *quantitativos*, podemos medir as diferenças numéricas entre os valores. Existem vários subtipo de dados quantitativos:

Para dados *intervalares*, podemos medir a distância (intervalo) entre pontos: *qual é a distância do valor A ao valor B? (A - B)*, sustentando afirmações como “A está a 12 unidades de distância de B”.

Para dados de *razão*, o ponto zero é significativo e, portanto, também podemos medir proporções ou fatores de escala: *o valor A é qual proporção do valor B? (A / B)*, sustentando afirmações como “A é 10% de B” ou “B é 7 vezes maior que A”.

No conjunto de dados acima, **`year`** é um campo quantitativo intervalar (o valor do ano "zero" é subjetivo), enquanto **`fertility`** e **`life_expect`** são campos quantitativos de razão (zero é significativo para calcular proporções). O Vega-Lite representa dados quantitativos, mas não faz uma distinção entre os tipos intervalar e de razão.

Valores quantitativos podem ser visualizados usando posição, tamanho ou valor de cor, entre outros canais. Um eixo com uma linha de base zero é essencial para comparações proporcionais de valores de razão, mas pode ser omitido com segurança para comparações intervalares.





### 2.2.4. Temporal (T)

Valores temporais medem pontos ou intervalos de tempo. Este tipo é um caso especial de valores quantitativos (marcadores de tempo) com semântica e convenções ricas (ou seja, o **Calendário Gregoriano**). O tipo temporal no Vega-Lite suporta o raciocínio sobre unidades de tempo (ano, mês, dia, hora, etc.) e fornece métodos para solicitar intervalos de tempo específicos.

Exemplos de valores temporais incluem strings de data como `"2019-01-04"` e `"04 Jan 2019"`, assim como datas e horas padronizadas, como o **ISO date-time format**: `"2019-01-04T17:50:35.643Z"`.

Não há valores temporais em nosso conjunto de dados de desenvolvimento global acima, pois o campo **`year`** é simplesmente codificado como um número inteiro. Para mais detalhes sobre o uso de dados temporais no Altair, consulte **Times and Dates documentation**.

### 2.2.5. Resumo

Esses tipos de dados não são mutuamente exclusivos, mas sim formam uma hierarquia: dados ordinais suportam comparações nominais (igualdade), enquanto dados quantitativos suportam comparações ordinais (ordem de classificação).

Além disso, esses tipos de dados *não* fornecem uma categorização fixa. Só porque um campo de dados é representado por um número, isso não significa que devemos tratá-lo como um tipo quantitativo! Por exemplo, podemos interpretar um conjunto de idades (10 anos, 20 anos, etc.) como nominal (menor de idade ou maior de idade), ordinal (agrupado por ano) ou quantitativo (calcular a idade média).

Agora vamos examinar como codificar visualmente esses tipos de dados!


## 2.3 Canais de Codificação

No coração do Altair está o uso de *codificações* que vinculam os campos de dados (com um tipo declarado de dado) aos *canais* de codificação de um tipo de *marca* escolhida. Neste notebook, examinaremos os seguintes canais de codificação:

- `x`: Posição horizontal (eixo x) da marca.
- `y`: Posição vertical (eixo y) da marca.
- `size`: Tamanho da marca. Pode corresponder a área ou comprimento, dependendo do tipo de marca.
- `color`: Cor da marca, especificada como uma [cor CSS válida](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value).
- `opacity`: Opacidade da marca, variando de 0 (totalmente transparente) a 1 (totalmente opaca).
- `shape`: Formato do símbolo plotado para marcas `point`.
- `tooltip`: Texto _Tooltip_ (de dica) exibido ao passar o mouse sobre a marca.
- `order`: Ordenação da marca, determina a ordem de pontos em curvas/área e a ordem de desenho.
- `column`: Faceta os dados em _subplots_ alinhados horizontalmente.
- `row`: Faceta os dados em _subplots_ alinhados verticalmente.

Para uma lista completa de canais disponíveis, veja [Documentação de codificação do Altair](https://altair-viz.github.io/user_guide/encodings/index.html).

### 2.3.1 X

O canal de codificação `x` estabelece a posição horizontal da marca (coordenada x). Além disso, escolhas padrão de eixos e títulos são feitas automaticamente. No gráfico abaixo, a escolha de um tipo de dado quantitativo resulta em uma escala de eixo linear contínua:

In [7]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q')
)

alt.Chart(...)

### 2.3.2 Y

O canal de codificação `y` estabelece a posição vertical da marca (coordenada y). Aqui, adicionamos o campo `cluster` usando um tipo de dado ordinal (`O`). O resultado é um eixo discreto que inclui uma banda dimensionada, com um tamanho de passo padrão, para cada valor único:

In [8]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:O')
)

alt.Chart(...)

_O que acontece com o gráfico acima se você trocar o tipo de dado `O` por `Q`?_

Se, em vez disso, adicionássemos o campo `life_expect` como uma variável quantitativa (`Q`), o resultado é um gráfico de dispersão com escalas lineares para ambos os eixos:

In [9]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q')
)

alt.Chart(...)

Por padrão, os eixos para escalas lineares quantitativas incluem o zero para garantir uma linha de base adequada para comparação de dados baseados em proporção. Em alguns casos, no entanto, um zero como linha de base pode ser irrelevante ou você pode querer focar em comparações de intervalos. Para desativar a inclusão automática do zero, configure o mapeamento da escala usando o atributo de codificação `scale`:

In [10]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q', scale=alt.Scale(zero=False)),
    alt.Y('life_expect:Q', scale=alt.Scale(zero=False))
)

alt.Chart(...)

Agora as escalas dos eixos não incluem mais o zero por padrão. Ainda resta algum espaçamento, visto que o domínio do eixo e os pontos finais são automaticamente ajustados para _bons_ (_nice_) números, como múltiplos de 5 ou 10.

_O que acontece se você também adicionar `nice=False` ao atributo `scale` acima?_

### 2.3.3. Tamanho
O canal de codificação `Size` define o tamanho ou a extensão de uma marca. O significado desse canal pode variar dependendo do tipo de marca. Para marcas de ponto, o canal `Size` mapeia a área em pixels do símbolo do gráfico, de modo que o diâmetro do ponto corresponda à raiz quadrada do valor do tamanho.

Vamos aprimorar nosso gráfico de dispersão codificando a população (pop) no canal `Size`. Como resultado, o gráfico agora também inclui uma legenda para interpretar os diferentes tamanhos.



In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q')
)

alt.Chart(...)

Em alguns casos, podemos ficar insatifeitos com o tamanho do intervalo padrão. Para definir um intervalo de tamanho personalizado, configuramos o parâmetro `range` do atributo `scale` em um vetor indicando os maiores e os menores tamanhos. Aqui nós iremos aprimorar o tamanho da codificação para o intervalo de 0 a 1000 pixels(valor máximo no domínio da escala):

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000]))
)

alt.Chart(...)

### 2.3.4. Cor e Opacidade

O canal de codificação `color` define a cor da marca. O estilo da codificação `color` depende fortemente do tipo de dado: dados nominais seguem, por padrão, um esquema de cores qualitativas multicolorido, enquanto dados ordinais e quantitativos utilizam gradientes de cores perceptualmente ordenados.

Aqui, codificamos o campo `cluster` usando o canal `color` e o tipo de dado nominal (`N`), o que resulta em uma matiz distinta para cada valor do cluster. Você consegue adivinhar o que o campo cluster pode indicar?

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N')
)

alt.Chart(...)

Se nós preferimos formatos preenchidos. Nós podemos passar um parâmetro `filled=True` para o método `mark_point`:

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N')
)

alt.Chart(...)

Por padrão, Altair usa um pouco de transparência para ajudar a combater sobreposição de pontos. Estamos livres para ajustar mais a opacidade, seja passando um valor padrão para o método `mark_*`, seja usando um canal de codificação dedicado.

Aqui demonstraremos como passar um valor constante para um canal de codificação em vez de vincular um campo de dado:

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5)
)

alt.Chart(...)

### 2.3.5. Formato

O canal de codificação `shape` define o formato geométrico usado pelas marcas de `point`. Diferente de outros canais que nós vimos até agora, o canal `shape` não pode ser usado por outros tipos de marca. Esse canal de codificação deve ser usado somente com dados nominais, já que não suporta ordenação perceptiva ou comparações de magnitude.

Vamos codificar o campo `cluster` usando tanto `shape`  como `color`. O uso de múltiplos canais para representar o mesmo campo de dados subjacente é conhecido como codificação redundante. O gráfico resultante combina as informações de cor e formato em um único símbolo na legenda:

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Shape('cluster:N')
)

alt.Chart(...)

### 2.3.6 Tooltips e Ordenação

Neste ponto, você pode estar se sentindo um pouco frustrado: construímos um gráfico, mas ainda não sabemos a que países os pontos visualizados correspondem! Vamos adicionar dicas de ferramenta interativas chamadas _tooltips_ (ferramenta de informações ou ferramenta de dicas) para possibilitar a exploração.

O canal de codificação `tooltip` determina o texto informativo a ser exibido quando o usuário move o cursor do mouse sobre um marcador. Vamos adicionar um tooltip para o campo de `country` e, em seguida, investigar quais países estão sendo representados.

In [17]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country')
)

alt.Chart(...)


Ao passar o mouse, você pode perceber que não é possível selecionar alguns dos pontos. Por exemplo, o maior círculo azul escuro corresponde à Índia, que está desenhado sobre um país com uma população menor, impedindo que o mouse passe por cima desse país. Para resolver esse problema, podemos usar o canal de codificação `order`.

O canal de codificação `order` determina a ordem dos pontos de dados, afetando tanto a ordem em que eles são desenhados quanto, para marcas de `line` e `area`, a ordem em que são conectados entre si.

Vamos ordenar os valores em ordem decrescente de classificação pela população (`pop`), garantindo que círculos menores sejam desenhados depois dos círculos maiores:

In [18]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending')
)

alt.Chart(...)


Agora podemos identificar o pequeno país que estava sendo obscurecido pela Índia: é Bangladesh!

Nós também podemos descobrir o que o campo `cluster` representa. Passe o mouse sobre os diversos pontos coloridos para formular sua própria explicação.

Neste ponto, adicionamos tooltips que mostram apenas uma única propriedade do registro de dados subjacente. Para mostrar múltiplos valores, podemos fornecer ao canal `tooltip` um array de codificações, uma para cada campo que queremos incluir:

In [19]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Order('pop:Q', sort='descending'),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('fertility:Q'),
        alt.Tooltip('life_expect:Q')
    ]   
)

alt.Chart(...)


Agora podemos ver múltiplos campos de dados ao passar o mouse por cima!

### 2.3.7 Facetas de Coluna e Linha

A posição espacial é um dos canais mais poderosos e flexíveis para codificação visual. Mas o que podemos fazer se já atribuímos campos aos canais x e y? Uma técnica valiosa é criar um gráfico treliça, que consiste em subgráficos que mostram um subconjunto dos dados. Um gráfico treliça é um exemplo da técnica mais geral de apresentar dados usando pequenos múltiplos ([small multiples](https://en.wikipedia.org/wiki/Small_multiple)) de visualizações.

Os canais de codificação `column` e `row` geram um conjunto horizontal (colunas) ou vertical (linhas) de subgráficos, nos quais os dados são particionados de acordo com o campo de dados fornecido.

Aqui está um gráfico treliça que divide os dados numa coluna por valor de `cluster`:

In [20]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending'),
    alt.Column('cluster:N')
)

alt.Chart(...)

O gráfico acima não cabe na tela, o que torna difícil comparar todos os subgráficos entre si! Podemos definir as propriedades padrão de largura (`width`) e altura (`height`) para criar um conjunto menor de múltiplos. Além disso, como os cabeçalhos das colunas já identificam os valores do `cluster`, podemos remover a legenda de cores (`color`) definindo-a como `None`. Para aproveitar melhor o espaço, também podemos orientar a legenda de tamanhos (`size`) para a parte inferior (`botom`) do gráfico.

In [21]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000]),
             legend=alt.Legend(orient='bottom', titleOrient='left')),
    alt.Color('cluster:N', legend=None),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending'),
    alt.Column('cluster:N')
).properties(width=135, height=135)

alt.Chart(...)


Por baixo dos panos, as codificações de colunas (`column`) e linhas (`row`) são traduzidas em uma nova especificação que utiliza o operador de composição de visualização `facet`. Vamos conhecer o conceito de facetas mais a fundo posteriormente!

Enquanto isso, *você consegue reescrever o gráfico acima para criar facetas em linhas em vez de colunas?*

### 2.3.8 Um Olhar Adiante: Filtragem Interativa

Nos módulos seguintes, exploraremos técnicas de interação para a exploração de dados. Aqui está uma prévia: vincular um controle deslizante de intervalo ao campo `year` para permitir a navegação interativa pelos dados de cada ano. Não se preocupe se o código abaixo parecer um pouco confuso neste momento, pois abordaremos interação em detalhes mais adiante.

*Arraste o controle deslizante para frente e para trás para ver como os valores dos dados mudam ao longo do tempo!*

In [22]:
select_year = alt.selection_single(
    name='select', fields=['year'], init={'year': 1955},
    bind=alt.binding_range(min=1955, max=2005, step=5)
)

alt.Chart(data).mark_point(filled=True).encode(
    alt.X('fertility:Q', scale=alt.Scale(domain=[0,9])),
    alt.Y('life_expect:Q', scale=alt.Scale(domain=[0,90])),
    alt.Size('pop:Q', scale=alt.Scale(domain=[0, 1200000000], range=[0,1000])),
    alt.Color('cluster:N', legend=None),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending')
).add_selection(select_year).transform_filter(select_year)

alt.Chart(...)

## 2.4 Marcas Gráficas

Nossa exploração dos canais de codificação acima utiliza exclusivamente marcas de pontos, os `point`, para visualizar os dados. No entanto, a marca do tipo `point`  é apenas uma das muitas formas geométricas que podem ser usadas para representar visualmente os dados. O Altair inclui vários tipos de marcas integradas, incluindo:

* `mark_area()` - Áreas preenchidas definidas por uma linha superior e uma linha de base.
* `mark_bar()` - Barras retangulares.
* `mark_circle()` - Pontos de dispersão como círculos preenchidos.
* `mark_line()` - Segmentos de linha conectados.
* `mark_point()` - Pontos de dispersão com formatos configuráveis.
* `mark_rect()` - Retângulos preenchidos, úteis para mapas de calor.
* `mark_rule()` - Linhas verticais ou horizontais que atravessam o eixo.
* `mark_square()` - Pontos de dispersão como quadrados preenchidos.
* `mark_text()` - Pontos de dispersão representados por texto.
* `mark_tick()` - Traços verticais ou horizontais.
Para uma lista completa, com links para exemplos, consulte a documentação de marcas do Altair. Em seguida, exploraremos alguns dos tipos de marcas mais comumente usados em gráficos estatísticos.

Para uma lista completa, com links para exemplos, consulte o [Altair marks documentation](https://altair-viz.github.io/user_guide/marks/index.html). Em seguida, exploraremos alguns dos tipos de marcas mais comumente usados em gráficos estatísticos.

### 2.4.1 Marcas de Pontos

A marca do tipo `point` representa pontos específicos, como em _gráficos de dispersão e gráficos de pontos_. Além dos canais de codificação `x` e `y` (para especificar as posições dos pontos em 2D), as marcas de ponto podem usar os canais `color`, `size` e `shape` para transmitir informações adicionais de campos de dados.

Abaixo está um gráfico de pontos de `fertility`, com o campo `cluster` codificado de forma redundante usando os canais `y` e `shape`.

In [23]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

alt.Chart(...)

Além dos canais de codificação, as marcas podem ser estilizadas fornecendo valores para os métodos `mark_*()`.

Por exemplo: as marcas de ponto são desenhadas com contornos traçados por padrão, mas podem ser especificadas para usar formas preenchidas com `filled`. Da mesma forma, é possível definir um valor padrão para `size` para ajustar a área total em pixels da marca de ponto.

In [24]:
alt.Chart(data2000).mark_point(filled=True, size=100).encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

alt.Chart(...)

### 2.4.2 Marcas de Círculo

A marca do tipo `circle` é uma abreviação conveniente para marcas `point` desenhadas como círculos preenchidos.

In [25]:
alt.Chart(data2000).mark_circle(size=100).encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

alt.Chart(...)

### 2.4.3 Marcas de Quadrado

O tipo de marca `square` é uma abreviação conveniente para marcas `point` desenhadas como quadrados preenchidos.

In [26]:
alt.Chart(data2000).mark_square(size=100).encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

alt.Chart(...)

### 2.4.4 Marcas de Traço

A marca do tipo `tick` representa um ponto de dados usando um pequeno segmento de linha ou "traço". Essas marcas são particularmente úteis para comparar valores em uma única dimensão com mínimo sobreposição. Um _gráfico de pontos_ desenhado com marcas de traço é, às vezes, chamado de _gráfico de faixa_.

In [27]:
alt.Chart(data2000).mark_tick().encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

alt.Chart(...)

### 2.4.5 Marcas de Barra

O tipo de marca `bar` desenha um retângulo com uma posição, largura e altura.

O gráfico abaixo é um gráfico de barras simples da população (`pop`) de cada país.

In [28]:
alt.Chart(data2000).mark_bar().encode(
    alt.X('country:N'),
    alt.Y('pop:Q')
)

alt.Chart(...)


A largura da barra é definida por um tamanho padrão. Vamos discutir como ajustar a largura da barra mais adiante neste notebook. (Um notebook subsequente analisará com mais detalhes como configurar os eixos, escalas e legendas.)

As barras também podem ser empilhadas. Vamos mudar a codificação de `x` para usar o campo `cluster`, e codificar `country` usando o canal `color`. Também desativaremos a legenda (que seria muito longa com as cores de todos os países!) e usaremos tooltips para o nome do país.

In [29]:
alt.Chart(data2000).mark_bar().encode(
    alt.X('cluster:N'),
    alt.Y('pop:Q'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
)

alt.Chart(...)


No gráfico acima, o uso do canal de codificação `color` faz com que o Altair / Vega-Lite empilhe automaticamente as marcas de barra. Caso contrário, as barras seriam desenhadas uma sobre a outra! Tente adicionar o parâmetro `stack=None` ao canal de codificação `y` para ver o que acontece se não aplicarmos o empilhamento...

Os exemplos acima criam gráficos de barras a partir de uma linha de base zero, e o canal `y` codifica apenas o valor (ou altura) não nulo da barra. No entanto, a marca de barra também permite especificar pontos de início e fim para transmitir intervalos.

O gráfico abaixo usa os canais `x` (ponto de início) e `x2` (ponto de fim) para mostrar o intervalo de expectativas de vida dentro de cada cluster regional. Abaixo, usamos as funções de agregação `min` e `max` para determinar os pontos finais do intervalo; discutiremos agregação com mais detalhes no próximo notebook!

Alternativamente, você pode usar `x` e `width` para fornecer um ponto de início mais um deslocamento, tal que `x2 = x + width`.

In [30]:
alt.Chart(data2000).mark_bar().encode(
    alt.X('min(life_expect):Q'),
    alt.X2('max(life_expect):Q'),
    alt.Y('cluster:N')
)

alt.Chart(...)

### 2.4.6 Marcas de Linha

O tipo de marca `line` conecta pontos plotados com segmentos de linha,por exemplo, para que a inclinação de uma linha transmita informações sobre a taxa de mudança.

Vamos criar um gráfico de linha de fertilidade por país ao passar dos anos, utilizando o conjunto de dados completo e nao filtrado de desenvolvimento global. Vamos esconder a legenda novamente e usar os tooltips em vez disso.

In [31]:
alt.Chart(data).mark_line().encode(
    alt.X('year:O'),
    alt.Y('fertility:Q'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width=400
)

alt.Chart(...)

Podemos ver variações interessantes por país, mas tendências gerais para números menores de crianças por família ao longo do tempo. Observe também que definimos uma largura customizada de 400 pixels. _Tente mudar (ou remover) as larguras e veja o que acontece!_

Vamos mudar alguns dos parâmetros de marca padrão para customizar o gráfico. Podemos definior o `strokeWidth` para determinar a espessura das linhas e o `opacity` para adicionar transparência. Por padrão, a marca `line` usa segmentos de linha reta para conectar pontos de dados. Em alguns casos, podemos querer suavizar as linhas. Podemos ajustar a interpolação usada para conectar pontos de dados definindo o parâmetro de marca `interpolate`. Vamos usar interpolação `monotone` para fornecer linhas suaves que também são garantidas para nao gerar advertidamente valores mínimos ou máximos "falsos" como resultado da interpolação.

In [32]:
alt.Chart(data).mark_line(
    strokeWidth=3,
    opacity=0.5,
    interpolate='monotone'
).encode(
    alt.X('year:O'),
    alt.Y('fertility:Q'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width=400
)

alt.Chart(...)

A marca `line` também pode ser usada para criar *gráficos de inclinação*, tabelas que destacam a mudança de valor entre dois pontos de comparação usando inclinações de linha.

Abaixo, vamos criar um gráfico de inclinação comparando as populações de cada país nos anos minímo e máximo em nosso conjunto de dados:1955 e 2005. Primeiro criamos um novo conjunto de dados Pandas filtrado para esses anos, então usar o Altaris para criar o gráfico de inclinação.

Por padrão, o Altair coloca os anos juntos. Para espaçar melhor os anos ao longo do eixo, podemos indicar o tamanho(em pixels) de passos discretos ao longo da largura do nosso gráfico, conforme indicado no comentário abaixo. Tente ajustar o valor da largura `step` abaixo e veja como o gráfico muda com a resposta.

In [ ]:
dataTime = data.loc[(data['year'] == 1955) | (data['year'] == 2005)]

alt.Chart(dataTime).mark_line(opacity=0.5).encode(
    alt.X('year:O'),
    alt.Y('pop:Q'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width={"step": 50} # ajusta o parâmetro step
)

alt.Chart(...)

### 2.4.7 Marcas de Área

O tipo de marca `area` combina aspectos de marcas `line` e `bar`: ele visualiza conexões (inclinações) entre pontos de dados, mas também mostra uma região preenchida, com uma aresta assumindo como padrão uma linha de base de valor zero.

O gráfico abaixo é um gráfico de área da população ao longo do tempo apenas para os Estados Unidos:

In [34]:
dataUS = data.loc[data['country'] == 'United States']

alt.Chart(dataUS).mark_area().encode(
    alt.X('year:O'),
    alt.Y('fertility:Q')
)

alt.Chart(...)

Semelhante as marcas `line`, as marcas `area` suportam o parametro `interpolate`.

In [35]:
alt.Chart(dataUS).mark_area(interpolate='monotone').encode(
    alt.X('year:O'),
    alt.Y('fertility:Q')
)

alt.Chart(...)

Semelhante às marcas `bar`, as marcas `area` também suportam empilhamento. Aqui, criamos um novo dataframe com dados para os três países da América do Norte, então os plotamos usando a marca `area` e um canal de codificação `color` para empilhar por país.

In [36]:
dataNA = data.loc[
    (data['country'] == 'United States') |
    (data['country'] == 'Canada') |
    (data['country'] == 'Mexico')
]

alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('pop:Q'),
    alt.Color('country:N')
)

alt.Chart(...)

Por padrão, o empilhamento é realizado em relação a uma linha de base zero. No entanto, outras opções `stack` estão disponíveis:

* `center` - empilhar em relação a uma linha de base no centro do gráfico, criando uma visualização de *gráfico de fluxo*, e
* `normalize` - para normalizar os dados somados em cada ponto de empilhamento para 100%, permitindo comparações percentuais.

Abaixo, adaptamos o gráfico definindo `y` o atributo de codificação `stack` para `center`. O que acontece se você defini-lo como `normalize`?

In [37]:
alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('pop:Q', stack='center'),
    alt.Color('country:N')
)

alt.Chart(...)


Para desabilitar o empilhamento completamente, defina o atributo `stack` como `None`. Também podemos adicionar `opacity` como um parâmetro de marca padrão para garantir que veremos as áreas sobrepostas!

In [38]:
alt.Chart(dataNA).mark_area(opacity=0.5).encode(
    alt.X('year:O'),
    alt.Y('pop:Q', stack=None),
    alt.Color('country:N')
)

alt.Chart(...)

O tipo de marca `area` também suporta linhas de base orientadas por dados, com as séries superior e inferior determinadas por campos de dados. Assim como com marcas `bar`, podemos usar os canais `x` e `x2` (ou `y` e `y2`) para fornecer pontos finais para a marca de área.

O gráfico abaixo visualiza a variação de fertilidade mínima e máxima, por ano, para os países da América do Norte:

In [39]:
alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('min(fertility):Q'),
    alt.Y2('max(fertility):Q')
).properties(
    width={"step": 40}
)

alt.Chart(...)

Podemos ver um alcance maior de valores em 1955, de pouco menos de 4 a pouco menos de 7. Em 2005, tanto os valores gerais de fertilidade quanto a variabilidade diminuíram, centralizados em torno de 2 filhos por família.

Todos os exemplos de marca `area` acima usam uma área orientada verticalmente. No entanto, Altair e Vega-Lite também suportam áreas horizontais. Vamos transpor o gŕafico acima, simplesmente trocando os canais `x` e `y`.

In [40]:
alt.Chart(dataNA).mark_area().encode(
    alt.Y('year:O'),
    alt.X('min(fertility):Q'),
    alt.X2('max(fertility):Q')
).properties(
    width={"step": 40}
)

alt.Chart(...)

## 2.5. Resumo

Concluímos nosso passeio pelos tipos de dados, canais de codificação e marcas gráficas! Agora você deve estar bem equipado para explorar ainda mais o espaço de codificações, tipos de marcas e parâmetros de marcas. Para uma referência abrangente - incluindo recursos que pulamos aqui! &ndash; consulte a documentação do Altair [marks](https://altair-viz.github.io/user_guide/marks.html) e [encoding](https://altair-viz.github.io/user_guide/encoding.html).

No próximo módulo, veremos o uso de transformações de dados para criar gráficos que resumem dados ou visualizam novos campos derivados. Em um módulo posterior, examinaremos como personalizar ainda mais seus gráficos modificando escalas, eixos e legendas.

Interessado em aprender mais sobre codificação visual?

<img title="Bertin's Taxonomy canais de codificação visual" src="https://cdn-images-1.medium.com/max/2000/1*jsb78Rr2cDy6zrE7j2IKig.png" style="max-width: 650px;"><br/>

<small>Bertin's taxonomia de codificações visuais de<a href="https://books.google.com/books/about/Semiology_of_Graphics.html?id=X5caQwAACAAJ"><em> Semiologia Gráfica</em></a>,  <a href="https://bost.ocks.org/mike/">Mike Bostock</a>.</small>

- O estudo sistemático das marcas, codificações visuais e tipos de dados subjacentes foi iniciado por [Jacques Bertin](https://en.wikipedia.org/wiki/Jacques_Bertin) em seu trabalho pioneiro de 1967, Sémiologie Graphique [_Sémiologie Graphique (The Semiology of Graphics)_](https://books.google.com/books/about/Semiology_of_Graphics.html?id=X5caQwAACAAJ). A imagem acima ilustra os canais de posição, tamanho, valor (brilho), textura, cor (matiz), orientação e forma, juntamente com as recomendações de Bertin sobre os tipos de dados que eles suportam.

- A estrutura de tipos de dados, marcas e canais também orienta as ferramentas automatizadas de design de visualização, começando com [Mackinlay's APT (A Presentation Tool)](https://scholar.google.com/scholar?cluster=10191273548472217907) em 1986, e continuando em sistemas mais recentes, como [Voyager](http://idl.cs.washington.edu/papers/voyager/) e [Draco](http://idl.cs.washington.edu/papers/draco/).

- A identificação de tipos nominais, ordinais, intervalares e proporcionais remonta, pelo menos, ao artigo de S. S. Stevens de 1947 [_On the theory of scales of measurement_](https://scholar.google.com/scholar?cluster=14356809180080326415).